<a href="https://colab.research.google.com/github/Challakrupajyothi/Rag-Pipeline/blob/main/CHAT_WITH_WEBSITE_USING_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CHAT WITH WEBSITE USING RAG**


In [1]:
pip install openai

In [2]:
pip install llama-index transformers sentence-transformers faiss-cpu requests beautifulsoup4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [3]:
pip install scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.5/254.5 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 6.4 MB/s eta 0:00:00


In [4]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.58.1
    Uninstalling openai-1.58.1:
      Successfully uninstalled openai-1.58.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-agent-openai 0.4.1 requires openai>=1.14.0, but you have openai 0.28.0 which is incompatible.
llama-index-embeddings-openai 0.3.1 requires openai>=1.1.0, but you have openai 0.28.0 which is incompatible.
llama-index-llms-openai 0.3.12 requires openai<2.0.0,>=1.58.1, but you have openai 0.28.0 which is incompatible.


In [6]:
import scrapy
from scrapy.crawler import CrawlerProcess
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import openai
import json
import os  # Import os for file checking

# Initialize the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Set your OpenAI API key
openai.api_key = ""  # Replace with your actual API key

# Function to chunk text
def chunk_text(text, chunk_size=200):
    words = text.split()
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# Function to query embedding
def query_embedding(query):
    return embedding_model.encode([query])[0]

# Function to perform similarity search in FAISS
def search_vector_database(query, k=5):
    query_vec = np.array([query_embedding(query)])
    distances, indices = index.search(query_vec, k)
    return indices, distances

# Function to generate response using OpenAI API
def generate_response_with_openai(query, context):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": f"Answer the following question based on the context: {context}\nQuestion: {query}"}
        ]
    )
    return response['choices'][0]['message']['content']

class UniversitySpider(scrapy.Spider):
    name = 'university_spider'
    start_urls = [
        "https://www.uchicago.edu/",
        "https://www.washington.edu/",
        "https://www.stanford.edu/",
        "https://und.edu/"
    ]

    def parse(self, response):
        # Extract text from paragraphs
        text = ' '.join(response.css('p::text').getall())
        yield {'url': response.url, 'content': text}

# Run the Scrapy spider only if scraped_data.json doesn't exist
if not os.path.exists('scraped_data.json'):
    process = CrawlerProcess(settings={
        'FEED_FORMAT': 'json',
        'FEED_URI': 'scraped_data.json',
    })

    process.crawl(UniversitySpider)
    process.start()  # The script will block here until the crawling is finished

# Load the scraped data
with open('scraped_data.json') as f:
    scraped_data = json.load(f)

# Chunk and embed data
chunked_data = {}
embeddings = []
metadata = []

for item in scraped_data:
    url = item['url']
    content = item['content']
    if content:
        chunks = chunk_text(content)
        chunk_embeddings = embedding_model.encode(chunks)
        embeddings.extend(chunk_embeddings)
        metadata.extend([{"url": url, "chunk": i} for i in range(len(chunks))])
        chunked_data[url] = chunks

embeddings = np.array(embeddings)

# Store embeddings in FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# User query
user_query = input("Enter your query: ")
retrieved_indices, _ = search_vector_database(user_query)

# Retrieve relevant chunks
retrieved_chunks = [metadata[i] for i in retrieved_indices[0]]
retrieved_text = "\n ".join([chunked_data[chunk['url']][chunk['chunk']] for chunk in retrieved_chunks])

# Generate response using OpenAI
response = generate_response_with_openai(user_query, retrieved_text)

# Print results
print("Retrieved Text:")
print(retrieved_text)
print("\nGenerated Response:")
print(response)

# Optionally, print both responses together
print("\n--- Summary ---")
print(f"General Response: {retrieved_text}")
print(f"Generated Response: {response}")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
2024-12-21 16:33:11 [sentence_transformers.SentenceTransformer] INFO: Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2024-12-21 16:33:11 [sentence_transformers.SentenceTransformer] INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/11" 200 0
2024-12-21 16:33:11 [urllib3.connectionpool] DEBUG: https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json HTTP/11" 200 0
2024-12-21 16:33:12 [urllib3.connectionpool] DEBUG: https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/re

Enter your query: SUMMARY


DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
2024-12-21 16:33:23 [openai] DEBUG: message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-4o-mini", "messages": [{"role": "user", "content": "Answer the following question based on the context: Project and Take Back the Night Rally educate and bring awareness to interpersonal violence. We provide qualified students with disabilities an equal opportunity to participate in all UND activities. Proud to be a Fighting Hawk? Use #UNDproud to be featured. Content is moderated. UND consistently ranks among the best for educational quality, affordability and career outcomes By clicking any link on this page you are giving your consent for us to set cookies, . 2024 University of North Dakota - Grand Forks, ND - Member of ND University System\\n Nobel Week wove stately traditions with imaginative recogniti

Retrieved Text:
Project and Take Back the Night Rally educate and bring awareness to interpersonal violence. We provide qualified students with disabilities an equal opportunity to participate in all UND activities. Proud to be a Fighting Hawk? Use #UNDproud to be featured. Content is moderated. UND consistently ranks among the best for educational quality, affordability and career outcomes By clicking any link on this page you are giving your consent for us to set cookies, . 2024 University of North Dakota - Grand Forks, ND - Member of ND University System
 Nobel Week wove stately traditions with imaginative recognitions. © 2024 University of Washington | Seattle, WA
 A diversity of people and ideas, coupled with free and open discourse, lays the foundation for students and scholars to bring forth original ideas that define fields and enrich human life. UChicago students develop the habits of mind and intellectual skills needed to confront complex challenges. UChicago researchers have